In [1]:
from pyspark.sql import SparkSession

In [2]:
SS = SparkSession.builder.appName('Test').master('local[1]').getOrCreate()

In [3]:
df = SS.read.csv("C:\\Users\Venkateswarlu.A\\Downloads\\NewYorkTaxiData\\yellow_tripdata_2015-01.csv",inferSchema=True, header=True)

In [4]:
print(df.schema)
print(df.take(5))

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('pickup_longitude', DoubleType(), True), StructField('pickup_latitude', DoubleType(), True), StructField('RateCodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('dropoff_longitude', DoubleType(), True), StructField('dropoff_latitude', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True)])
[Row(VendorID=2

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format, year, month, col
df = df.withColumn("pickup_month", month (col("tpep_pickup_datetime")))

In [6]:
print(df.schema)
df.write.mode('append').option("header",True).partitionBy("pickup_month").format("csv").save("C:\\Users\Venkateswarlu.A\\Downloads\\NewYorkTaxiData\\PARTITIONED_DATA\\")

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('pickup_longitude', DoubleType(), True), StructField('pickup_latitude', DoubleType(), True), StructField('RateCodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('dropoff_longitude', DoubleType(), True), StructField('dropoff_latitude', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('pic

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, col, dayofmonth
SS = SparkSession.builder.appName('Test').master('local[1]').getOrCreate()
def partLoad (input_file, storage_path, part_column, part_by_values, is_date_column):
    part_renamed_cols = list([])
    if isinstance(part_column,list):
        print("Enter Single Column for partitioning for now!")
    else:
        if is_date_column == 'Yes':
            if isinstance(part_by_values,list) and  not set(part_by_values).issubset(["year","month","day"]):
                print("input type of part_by_values is " + str(type(part_by_values)))
                print(part_by_values)
                print("Enter year, month or day in the part_by_values parameter.")
            else:
                df = SS.read.csv(input_file, inferSchema=True, header=True)
                for col_name in part_by_values:
                    part_col_name = 'part_'+col_name
                    if col_name == "year":
                        df = df.withColumn(part_col_name, year(col(part_column)))
                        part_renamed_cols.append(part_col_name)
                        print(df.show())
                    elif col_name == "month":
                        df = df.withColumn(part_col_name, month(col(part_column)))
                        part_renamed_cols.append(part_col_name)
                        print(df.show())
                    else:
                        df = df.withColumn(part_col_name, dayofmonth(col(part_column)))
                        part_renamed_cols.append(part_col_name)
                print(part_renamed_cols)
                print('Entering to partition data now.')
                df.write.mode('append').option("header", True).partitionBy(part_renamed_cols).format("csv").save(storage_path)

partLoad("C:\\Users\Venkateswarlu.A\\Downloads\\NewYorkTaxiData\\yellow_tripdata_2016-03.csv",
          "C:\\Users\Venkateswarlu.A\\Downloads\\NewYorkTaxiData\\PARTITIONED_DATA\\",
          "tpep_pickup_datetime",
          ["year","month"],
          "Yes")

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|part_year|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+---------+
|       1| 2016-03-01 00:00:00|  2016-03-01 00:07:55|              1|          2.5|-73.97674560546875| 40.